In [3]:
!pip install pulp

   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB 435.7 kB/s eta 0:00:41
   ---------------------------------------- 0.1/17.7 MB 573.4 kB/s eta 0:00:31
   ---------------------------------------- 0.2/17.7 MB 893.0 kB/s eta 0:00:20
    --------------------------------------- 0.3/17.7 MB 1.2 MB/s eta 0:00:15
   - -------------------------------------- 0.5/17.7 MB 1.9 MB/s eta 0:00:09
   - -------------------------------------- 0.7/17.7 MB 2.4 MB/s eta 0:00:08
   -- ------------------------------------- 1.2/17.7 MB 3.4 MB/s eta 0:00:05
   ----- ---------------------------------- 2.2/17.7 MB 5.5 MB/s eta 0:00:03
   ------- -------------------------------- 3.2/17.7 MB 7.1 MB/s eta 0:00:03
   ----------- ---------------------------- 5.0/17.7 MB 9.9 MB/s eta 0:00:02
   ------------- -------------------------- 6.2/17.7 MB 11.2 MB/s eta 0:00:02
   ---

In [1]:
import pandas as pd
import numpy as np
import pulp

In [2]:
df = pd.read_csv("final.csv")

C:\Users\kopal\AppData\Local\Temp\ipykernel_24096\4268510692.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("final.csv")


In [3]:
df

,Unnamed: 0,Year,FlightDate,DOT_ID_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginCityName,OriginWac,DestAirportID,...,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,19038,2024,2024-01-15,20409,N945JT,616,14771,"San Francisco, CA",91,12478,...,10.6,7.2,15.0,0.0,NaN,54.0,9.0,NaN,1020.4,NaN
1,19039,2024,2024-01-28,19805,N990AU,341,11298,"Dallas/Fort Worth, TX",74,14492,...,7.8,1.1,16.7,0.0,0.0,282.0,10.1,NaN,1028.9,NaN
2,19040,2024,2024-01-02,19393,N7884G,3257,10821,"Baltimore, MD",35,14107,...,5.5,2.8,7.8,0.0,0.0,310.0,15.4,NaN,1019.9,NaN
3,19041,2024,2024-01-07,20304,N311SY,3848,13487,"Minneapolis, MN",63,13232,...,-4.3,-5.0,-1.7,0.0,0.0,317.0,10.2,NaN,1015.0,NaN
4,19042,2024,2024-01-31,20452,N642RW,3493,12264,"Washington, DC",38,10785,...,4.9,2.8,6.7,0.0,0.0,33.0,6.8,NaN,1019.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546943,548681,2022,2022-12-20,20363,N293PQ,5136,10397,"Atlanta, GA",34,14814,...,0.9,-7.2,12.8,0.3,0.0,26.0,7.3,NaN,1020.1,NaN
546944,548682,2022,2022-12-07,20416,N649NK,144,13796,"Oakland, CA",91,12889,...,12.1,10.6,13.9,0.3,0.0,223.0,8.7,NaN,1021.9,NaN
546945,548683,2022,2022-12-10,19393,N492WN,3183,10693,"Nashville, TN",54,11259,...,5.7,4.4,7.2,5.8,0.0,1.0,2.2,NaN,1018.4,NaN
546946,548684,2022,2022-12-21,19393,N8727M,1364,12402,"Hilo, HI",2,12173,...,-16.4,-25.6,10.6,6.4,127.0,35.0,20.2,NaN,1029.2,NaN


In [4]:
df = df.drop(['Unnamed: 0', 'Year', 'DepartureDelayGroups', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'DivAirportLandings', 'wpgt', 'tsun'], axis = 1)

In [6]:
df = df.drop(['index'], axis = 1)
df.columns

Index(['FlightDate', 'DOT_ID_Reporting_Airline', 'Tail_Number',
       'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginCityName',
       'OriginWac', 'DestAirportID', 'DestCityName', 'DestWac', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime',
       'AirTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'CancellationCode_encoded', 'tavg', 'tmin', 'tmax', 'prcp', 'snow',
       'wdir', 'wspd', 'pres'],
      dtype='object')

In [7]:
grouped = df.groupby(['FlightDate', 'OriginCityName'])

# Fill NaN in Col1 with the mean of matching groups
for col in ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres']:
    df[col] = grouped[col].transform(lambda x: x.fillna(x.mean()))

In [8]:
df.isna().sum()

FlightDate                              0
DOT_ID_Reporting_Airline                0
Tail_Number                          1989
Flight_Number_Reporting_Airline         0
OriginAirportID                         0
OriginCityName                          0
OriginWac                               0
DestAirportID                           0
DestCityName                            0
DestWac                                 0
CRSDepTime                              0
DepTime                              9945
DepDelayMinutes                      9953
CRSArrTime                              0
ArrTime                             10419
ArrDelayMinutes                     11676
Diverted                                0
CRSElapsedTime                          0
ActualElapsedTime                   11676
AirTime                             11676
Flights                                 0
Distance                                0
CarrierDelay                       431997
WeatherDelay                      

In [9]:
df = df.drop(['snow'], axis=1)
df = df.dropna()
df

,FlightDate,DOT_ID_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginCityName,OriginWac,DestAirportID,DestCityName,DestWac,...,SecurityDelay,LateAircraftDelay,CancellationCode_encoded,tavg,tmin,tmax,prcp,wdir,wspd,pres
4,2024-01-31,20452,N642RW,3493,12264,"Washington, DC",38,10785,"Burlington, VT",16,...,0.0,0.0,0,4.9,2.8,6.7,0.0,33.0,6.8,1019.2
5,2024-01-15,19790,N590NW,1204,11433,"Detroit, MI",43,14635,"Fort Myers, FL",33,...,0.0,58.0,0,-15.3,-18.5,-11.9,0.0,234.0,21.8,1023.6
7,2024-01-05,20368,333NV,292,14262,"Palm Springs, CA",91,10666,"Bellingham, WA",93,...,0.0,1.0,0,13.8,5.6,21.7,0.0,326.0,8.5,1020.4
10,2024-01-13,19790,N921AT,2459,12197,"White Plains, NY",22,14027,"West Palm Beach/Palm Beach, FL",33,...,0.0,0.0,0,7.9,0.6,13.9,35.3,226.0,21.8,995.3
17,2024-01-09,19393,N255WN,1670,14986,"Sarasota/Bradenton, FL",33,11259,"Dallas, TX",74,...,0.0,366.0,0,21.2,17.2,25.0,7.2,178.0,30.8,1010.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546904,2022-12-24,20304,N730EV,3226,13930,"Chicago, IL",41,12197,"White Plains, NY",22,...,0.0,0.0,0,22.9,20.0,27.0,0.2,186.0,16.9,1015.3
546907,2022-12-01,20397,N592NN,5087,11278,"Washington, DC",38,13198,"Kansas City, MO",64,...,0.0,21.0,0,-7.5,-20.5,4.4,4.1,237.0,29.2,1002.0
546917,2022-12-29,20416,N921NK,2410,11618,"Newark, NJ",21,13796,"Oakland, CA",91,...,0.0,14.0,0,19.7,16.6,23.1,0.0,43.0,17.1,1023.8
546921,2022-12-23,19790,N897DN,2554,12892,"Los Angeles, CA",91,13303,"Miami, FL",33,...,0.0,32.0,0,23.1,21.0,27.0,0.1,170.0,14.1,1021.4


In [14]:
df.isna().sum()

FlightDate                         0
DOT_ID_Reporting_Airline           0
Tail_Number                        0
Flight_Number_Reporting_Airline    0
OriginAirportID                    0
OriginCityName                     0
OriginWac                          0
DestAirportID                      0
DestCityName                       0
DestWac                            0
CRSDepTime                         0
DepTime                            0
DepDelayMinutes                    0
CRSArrTime                         0
ArrTime                            0
ArrDelayMinutes                    0
Diverted                           0
CRSElapsedTime                     0
ActualElapsedTime                  0
AirTime                            0
Flights                            0
Distance                           0
CarrierDelay                       0
WeatherDelay                       0
NASDelay                           0
SecurityDelay                      0
LateAircraftDelay                  0
C

In [15]:
df[['ArrDelayMinutes', 'Diverted', 'ActualElapsedTime', 'AirTime', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'CancellationCode_encoded']] = df[['ArrDelayMinutes', 'Diverted', 'ActualElapsedTime', 'AirTime', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'CancellationCode_encoded']].fillna(0)

In [16]:
df['DateObjects'] = pd.to_datetime(df['FlightDate'])

In [17]:
df['DateObjects'] = pd.to_datetime(df['FlightDate'])

In [18]:
df = df.drop(['FlightDate'], axis = 1)

In [123]:
cut_off_date = pd.to_datetime('2024-08-01')

In [125]:
test_df = df[df['DateObjects'] >= cut_off_date]

In [127]:
test_df

,DOT_ID_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginCityName,OriginWac,DestAirportID,DestCityName,DestWac,CRSDepTime,...,LateAircraftDelay,CancellationCode_encoded,tavg,tmin,tmax,prcp,wdir,wspd,pres,DateObjects
361110,20409,N3139J,720,10397,"Atlanta, GA",34,12478,"New York, NY",22,1535,...,0.0,0,25.6,21.1,32.8,3.0,188.0,10.1,1021.3,2024-08-01
370393,20416,N607NK,1101,11057,"Charlotte, NC",36,13204,"Orlando, FL",33,1952,...,0.0,0,26.5,18.3,33.9,0.0,115.0,6.5,1022.6,2024-08-01
357266,19805,N771XF,2820,14100,"Philadelphia, PA",23,10721,"Boston, MA",13,1711,...,124.0,0,18.8,15.0,24.4,0.0,225.0,6.8,1016.5,2024-08-01
375414,20398,N305CL,3594,15919,"Fayetteville, AR",71,11057,"Charlotte, NC",36,1823,...,124.0,0,29.8,24.7,36.0,0.0,303.0,6.4,1016.6,2024-08-01
361067,19805,N874NN,1898,13303,"Miami, FL",33,10397,"Atlanta, GA",34,2238,...,0.0,0,19.2,16.7,23.0,0.1,164.0,15.4,1020.0,2024-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358053,19805,N157AA,813,11298,"Dallas/Fort Worth, TX",74,11618,"Newark, NJ",21,1133,...,27.0,0,30.5,24.7,35.4,0.3,218.0,10.8,1013.8,2024-08-31
360579,19790,N608AT,2681,10397,"Atlanta, GA",34,13485,"Madison, WI",45,1159,...,0.0,0,29.4,23.3,33.9,0.0,123.0,13.9,1015.4,2024-08-31
372777,19790,N572DT,820,11278,"Washington, DC",38,14869,"Salt Lake City, UT",87,1730,...,0.0,0,22.7,18.9,28.2,3.2,176.0,13.1,1014.7,2024-08-31
367296,19805,N807AW,615,15304,"Tampa, FL",33,13303,"Miami, FL",33,1000,...,14.0,0,32.2,23.9,39.4,0.0,188.0,10.4,1010.4,2024-08-31


In [129]:
test_df['CancellationCode_encoded'] = test_df['CancellationCode_encoded'].apply(lambda x: 1 if x >= 1 else x)

C:\Users\kopal\AppData\Local\Temp\ipykernel_24096\1526038971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['CancellationCode_encoded'] = test_df['CancellationCode_encoded'].apply(lambda x: 1 if x >= 1 else x)


In [131]:
test_df.columns

Index(['DOT_ID_Reporting_Airline', 'Tail_Number',
       'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginCityName',
       'OriginWac', 'DestAirportID', 'DestCityName', 'DestWac', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime',
       'AirTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'CancellationCode_encoded', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir',
       'wspd', 'pres', 'DateObjects'],
      dtype='object')

In [133]:
test_df = test_df.drop(columns = ['OriginAirportID', 'OriginWac', 'DestAirportID', 'DestWac', 'DepTime', 'ArrTime',
       'ArrDelayMinutes', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime',
       'AirTime', 'Flights', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay','tavg', 'tmin', 'tmax', 'prcp', 'wdir',
       'wspd', 'pres', ], axis = 1)

In [135]:
test_df = test_df.drop(columns = ['CancellationCode_encoded'], axis = 1)

In [137]:
test_df

,DOT_ID_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginCityName,DestCityName,CRSDepTime,DepDelayMinutes,CRSArrTime,Distance,DateObjects
361110,20409,N3139J,720,"Atlanta, GA","New York, NY",1535,0.0,1804,760,2024-08-01
370393,20416,N607NK,1101,"Charlotte, NC","Orlando, FL",1952,53.0,2145,468,2024-08-01
357266,19805,N771XF,2820,"Philadelphia, PA","Boston, MA",1711,124.0,1834,280,2024-08-01
375414,20398,N305CL,3594,"Fayetteville, AR","Charlotte, NC",1823,135.0,2144,754,2024-08-01
361067,19805,N874NN,1898,"Miami, FL","Atlanta, GA",2238,65.0,39,594,2024-08-01
...,...,...,...,...,...,...,...,...,...,...
358053,19805,N157AA,813,"Dallas/Fort Worth, TX","Newark, NJ",1133,32.0,1602,1372,2024-08-31
360579,19790,N608AT,2681,"Atlanta, GA","Madison, WI",1159,52.0,1259,707,2024-08-31
372777,19790,N572DT,820,"Washington, DC","Salt Lake City, UT",1730,0.0,2015,1851,2024-08-31
367296,19805,N807AW,615,"Tampa, FL","Miami, FL",1000,22.0,1109,204,2024-08-31


In [139]:
prob = pulp.LpProblem("Flight_Rescheduling", pulp.LpMinimize)

# Decision variables
cancel = pulp.LpVariable.dicts("cancel", test_df.index, cat='Binary')
rescheduled_departure = pulp.LpVariable.dicts("rescheduled_departure", test_df.index, lowBound=0, cat='Continuous')

# Objective function: Minimize total delay and cancellations
penalty_cancel = 1000
prob += (
    pulp.lpSum([rescheduled_departure[i] - test_df.loc[i, 'CRSDepTime'] + test_df.loc[i, 'DepDelayMinutes']
                for i in test_df.index]) +
    penalty_cancel * pulp.lpSum([cancel[i] for i in test_df.index])
)

In [140]:
for i in test_df.index:
    for j in test_df.index:
        if (
            i != j 
            and test_df.loc[i, 'Tail_Number'] == test_df.loc[j, 'Tail_Number']  # Same aircraft
            and test_df.loc[i, 'DestCityName'] == test_df.loc[j, 'OriginCityName']  # Dest to origin match
            and test_df.loc[j, 'DateObjects'] >= test_df.loc[i, 'DateObjects']  # Same or next day
        ):
            prob += (
                rescheduled_departure[j] >=
                rescheduled_departure[i] + (test_df.loc[i, 'CRSArrTime'] - test_df.loc[i, 'CRSDepTime']) * (1 - cancel[i])
            )

# 2. Flight connection: A flight can only depart after the arrival of the connecting flight.
for i in test_df.index:
    for j in test_df.index:
        if i != j and test_df.loc[i, 'DestCityName'] == test_df.loc[j, 'OriginCityName']:
            prob += rescheduled_departure[j] >= test_df.loc[i, 'CRSArrTime'] + test_df.loc[i, 'DepDelayMinutes']


In [142]:
for i in test_df.index:
    prob += rescheduled_departure[i] >= test_df.loc[i, 'CRSDepTime']

# Solve the optimization problem
prob.solve()

# Output the results
print(f"Status: {pulp.LpStatus[prob.status]}")
for i in test_df.index:
    if pulp.value(cancel[i]) == 1:
        print(f"Flight from {test_df.loc[i, 'OriginCityName']} to {test_df.loc[i, 'DestCityName']} is cancelled.")
    else:
        new_departure = pulp.value(rescheduled_departure[i])
        print(f"Flight from {test_df.loc[i, 'OriginCityName']} to {test_df.loc[i, 'DestCityName']} departs at {new_departure:.2f}.")

Status: Optimal
Flight from Atlanta, GA to New York, NY departs at 3288.00.
Flight from Charlotte, NC to Orlando, FL departs at 2887.00.
Flight from Philadelphia, PA to Boston, MA departs at 2416.00.
Flight from Fayetteville, AR to Charlotte, NC departs at 2329.00.
Flight from Miami, FL to Atlanta, GA departs at 2753.00.
Flight from Atlanta, GA to Salt Lake City, UT departs at 3288.00.
Flight from Atlanta, GA to Newark, NJ departs at 3288.00.
Flight from Chicago, IL to Lexington, KY departs at 2858.00.
Flight from New York, NY to Cincinnati, OH departs at 3019.00.
Flight from Denver, CO to San Francisco, CA departs at 2875.00.
Flight from Baltimore, MD to Cleveland, OH departs at 2412.00.
Flight from Rochester, NY to Newark, NJ departs at 2219.00.
Flight from San Juan, PR to New York, NY departs at 2430.00.
Flight from Salt Lake City, UT to San Diego, CA departs at 2461.00.
Flight from Sacramento, CA to Phoenix, AZ departs at 2462.00.
Flight from New York, NY to Oklahoma City, OK depar

In [143]:
for i in test_df.index:
    new_departure = pulp.value(rescheduled_departure[i])
    print(f"Flight {test_df.loc[i, 'Flight_Number_Reporting_Airline']} rescheduled to depart at {new_departure:.2f} minutes.")

# Evaluate results
total_delay = sum(
    pulp.value(rescheduled_departure[i]) - test_df.loc[i, 'CRSDepTime'] + test_df.loc[i, 'DepDelayMinutes']
    for i in test_df.index
)
print(f"Total Delay: {total_delay:.2f} minutes")

total_fuel = sum(
    1 * test_df.loc[i, 'Distance'] for i in test_df.index
)
print(f"Total Fuel Consumption: {total_fuel:.2f} units")

Flight 720 rescheduled to depart at 3288.00 minutes.
Flight 1101 rescheduled to depart at 2887.00 minutes.
Flight 2820 rescheduled to depart at 2416.00 minutes.
Flight 3594 rescheduled to depart at 2329.00 minutes.
Flight 1898 rescheduled to depart at 2753.00 minutes.
Flight 423 rescheduled to depart at 3288.00 minutes.
Flight 1595 rescheduled to depart at 3288.00 minutes.
Flight 3781 rescheduled to depart at 2858.00 minutes.
Flight 5023 rescheduled to depart at 3019.00 minutes.
Flight 2267 rescheduled to depart at 2875.00 minutes.
Flight 450 rescheduled to depart at 2412.00 minutes.
Flight 3523 rescheduled to depart at 2219.00 minutes.
Flight 1004 rescheduled to depart at 2430.00 minutes.
Flight 2914 rescheduled to depart at 2461.00 minutes.
Flight 2390 rescheduled to depart at 2462.00 minutes.
Flight 4526 rescheduled to depart at 3019.00 minutes.
Flight 2245 rescheduled to depart at 3019.00 minutes.
Flight 3126 rescheduled to depart at 2511.00 minutes.
Flight 815 rescheduled to depar

In [144]:
test_df['DepDelayMinutes'].sum()

306241.0

In [145]:
np.random.seed(42)
test_df['stochastic_delay'] = test_df['DepDelayMinutes'] + np.random.normal(0, 10, len(test_df))

# Initialize the optimization problem
prob = pulp.LpProblem("Flight_Rescheduling", pulp.LpMinimize)

# Decision variables
cancel = pulp.LpVariable.dicts("cancel", test_df.index, cat='Binary')
rescheduled_departure = pulp.LpVariable.dicts("rescheduled_departure", test_df.index, lowBound=0, cat='Continuous')

# Objective function: Minimize stochastic delays and cancellations
penalty_cancel = 1000
prob += (
    pulp.lpSum([rescheduled_departure[i] - test_df.loc[i, 'CRSDepTime'] + test_df.loc[i, 'stochastic_delay']
                for i in test_df.index]) +
    penalty_cancel * pulp.lpSum([cancel[i] for i in test_df.index])
)


In [146]:
for i in test_df.index:
    prob += rescheduled_departure[i] >= test_df.loc[i, 'CRSDepTime']
    prob += rescheduled_departure[i] >= test_df.loc[i, 'CRSDepTime']

# Solve the optimization problem
prob.solve()

# Output the results
for i in test_df.index:
    if pulp.value(cancel[i]) == 1:
        print(f"Flight {test_df.loc[i, 'Flight_Number_Reporting_Airline']} is cancelled.")
    else:
        new_departure = pulp.value(rescheduled_departure[i])
        print(f"Flight departs at {new_departure:.2f} minutes with delay.")

Flight departs at 1535.00 minutes with delay.
Flight departs at 1952.00 minutes with delay.
Flight departs at 1711.00 minutes with delay.
Flight departs at 1823.00 minutes with delay.
Flight departs at 2238.00 minutes with delay.
Flight departs at 1400.00 minutes with delay.
Flight departs at 1200.00 minutes with delay.
Flight departs at 1946.00 minutes with delay.
Flight departs at 1756.00 minutes with delay.
Flight departs at 1900.00 minutes with delay.
Flight departs at 1710.00 minutes with delay.
Flight departs at 1018.00 minutes with delay.
Flight departs at 2159.00 minutes with delay.
Flight departs at 1145.00 minutes with delay.
Flight departs at 1635.00 minutes with delay.
Flight departs at 1920.00 minutes with delay.
Flight departs at 1759.00 minutes with delay.
Flight departs at 1713.00 minutes with delay.
Flight departs at 1314.00 minutes with delay.
Flight departs at 2029.00 minutes with delay.
Flight departs at 1800.00 minutes with delay.
Flight departs at 2030.00 minutes 

In [147]:
baseline_delay = test_df['DepDelayMinutes'].sum()
optimized_delay = sum([pulp.value(rescheduled_departure[i]) - test_df.loc[i, 'CRSDepTime']
                       for i in test_df.index])
print(f"Baseline Total Delay: {baseline_delay}")
print(f"Optimized Total Delay: {optimized_delay}")
print(f"Improvement: {baseline_delay - optimized_delay} minutes")

Baseline Total Delay: 306241.0
Optimized Total Delay: 0.0
Improvement: 306241.0 minutes


In [148]:
prob = pulp.LpProblem("Flight_Rescheduling", pulp.LpMinimize)

# Decision variables
rescheduled_departure = pulp.LpVariable.dicts(
    "rescheduled_departure", test_df.index, lowBound=0, cat='Continuous'
)
cancel = pulp.LpVariable.dicts("cancel", test_df.index, cat='Binary')

# Objective: Minimize delay, cancellations, and fuel usage
penalty_cancel = 1000  # Arbitrary penalty for cancellations
fuel_rate = 0.1  # Arbitrary fuel cost per km

prob += (
    pulp.lpSum([
        rescheduled_departure[i] - test_df.loc[i, 'CRSDepTime'] +
        test_df.loc[i, 'DepDelayMinutes'] + penalty_cancel * cancel[i] +
        fuel_rate * test_df.loc[i, 'Distance']
        for i in test_df.index
    ])
)

In [149]:
for i in test_df.index:
    prob += rescheduled_departure[i] >= test_df.loc[i, 'CRSDepTime'] * (1 - cancel[i])

# 2. Turnaround time constraint: Flights using the same aircraft must have a gap
turnaround_time = 60  # 1 hour minimum turnaround
for i, j in zip(test_df.index[:-1], test_df.index[1:]):
    if test_df.loc[i, 'Tail_Number'] == test_df.loc[j, 'Tail_Number']:
        prob += rescheduled_departure[j] >= rescheduled_departure[i] + turnaround_time

# 3. Connecting flight constraint: Ensure connecting flights are delayed appropriately
connections = [(i, i + 1) for i in range(600)]  # Example: Flight 1 must connect to Flight 2
for item in connections:
    i, j = item
    prob += rescheduled_departure[j] >= rescheduled_departure[i]

# Solve the optimization problem
prob.solve()

KeyError: 1

In [ ]:
for i in test_df.index:
    if pulp.value(cancel[i]) == 1:
        print(f"Flight {test_df.loc[i, 'Flight_Number_Reporting_Airline']} is cancelled.")
    else:
        new_departure = pulp.value(rescheduled_departure[i])
        print(f"Flight {test_df.loc[i, 'Flight_Number_Reporting_Airline']} departs at {new_departure:.2f} minutes.")


In [ ]:
total_delay = sum(
    pulp.value(rescheduled_departure[i]) - test_df.loc[i, 'CRSDepTime']
    for i in test_df.index if pulp.value(cancel[i]) == 0
)
print(f"Total Delay: {total_delay} minutes")